In [ ]:
-- IO actions are also a instance of Functor

-- instance Functor IO where  
--     fmap f action = do  
--         result <- action  
--         return (f result)

-- we get the value out of the io action by binding it to result
-- then put it back with return after doing f on result

do
    reversed <- fmap reverse $ readFile "foo.txt"
    putStrLn reversed

In [ ]:
-- Another instace of Functor is (->) r
-- (->) r looks confusing but its the same as (+) 2 3. 
-- We prefix the binary operator which in this case is the "function arrow"
-- This allows use to fmap over functions
-- its implmentation looks somthing like this:

-- instance Functor ((->) r) where  
--     fmap f g = (\x -> f (g x))  

-- fmaps type is: fmap :: (a -> b) -> f a -> f b
-- now if we replace the f's with are type parameter
-- fmaps type is: fmap :: (a -> b) -> ((->) r a) -> ((->) r b)
-- and if we make those prefix operators infix
-- fmaps type is: fmap :: (a -> b) -> (r -> a) -> (r -> b)

-- so fmap'ing over functions is actually just function composition
(*3) . (+100) $ 10
(*3) `fmap` (+100) $ 10

-- Since functor takes a single type parameter we 

In [ ]:
-- Functors have some laws which should be followed

-- if maping the identity function (such as \x -> x) 
-- the result should be the same as the original Functir

-- if maping a function composition the fmap (f . g)
-- the result should be the same as maping over one function then the next fmap f (fmap g)

In [ ]:
import Control.Applicative
-- an applicative functor is defined as so:

-- class (Functor f) => Applicative f where  
--     pure :: a -> f a  
--     (<*>) :: f (a -> b) -> f a -> f b  

-- pure takes a value and returns a minimal context that returns that value
-- <*> is like fmap however it takes a functor and applies the function 'inside' it
-- to the other functor

-- heres how its implemented for the maybe type

-- instance Applicative Maybe where  
--     pure = Just  
--     Nothing <*> _ = Nothing  
--     (Just f) <*> something = fmap f something

-- pure = Just is the same as pure x = Just x, it wraps a value in the maybe type
-- their is no function in nothing, so return nothing
-- their can be a function in a just type, so ew extract it and apply it to the functor somthing

Just (+3) <*> Just 9
pure (+3) <*> Just 10

In [ ]:
-- Control.Applicative also exports a <$> operator
-- which is just fmap, which allows us to do things like:

(++) <$> Just "Alan " <*> Just "Turing"

-- the first fmap turns (Just "Ben ") into (Just "Ben ++")
-- the second applicative fmap turns (Just "Ben ++" into Just "Ben Sheffield")

(*) <$> [1, 2, 3] <*> [4, 5, 6]

In [ ]:
-- What if instead of applying the function to all combinations of elements
-- instead we required it to be applied to each element in the same position
-- i.e. 

-- (*) <$> [1, 2, 3] <*> [4, 5, 6] = [4, 10, 18]

-- We can use ZipLists for this, they wrap a list so that we can do just this
let foo = (*) <$> ZipList [1, 2, 3] <*> ZipList [4, 5, 6]
foo

-- If we want to extract the list we need to call getZipList
getZipList foo

In [ ]:
-- newtype is the prefered way to create a wrapper type
-- data will require unwrapping/wrapping at runtime whereas
-- haskell doesnt actually box are type if we use newtype.

-- an example would be ziplist

-- newtype ZipList a = ZipList { getZipList :: [a] }

-- this would be exactally the same if we replaced newtype with data
-- (except for runtime performace)


In [ ]:
-- * is assosiative and has the identity of 1
4 * (5 * 6)
(4 * 5) * 6
4 * 1

-- ++ is also assosiative and has the identity []
[1,2,3] ++ ([4,5]++[6,7])
([1,2,3] ++ [4,5])++[6,7]
[1,2,3] ++ []

-- these are monoids
import Data.Monoid

-- and their definition is as follows:

-- class Monoid m where  
--     mempty :: m  
--     mappend :: m -> m -> m  
--     mconcat :: [m] -> m  
--     mconcat = foldr mappend mempty

-- mempty is the identity value of the monoid
-- mappend takes two monoids and applys the binary function
-- mconcat reduces a list of monoids by applying mappend

-- lists are monoid's as we showed before

-- instance Monoid [a] where  
--     mempty = []  
--     mappend = (++)  

mconcat [[1,2],[3..6],[7]]

-- another usefull monoid is maybe
Just "bob" `mappend` Nothing
Just "bob" `mappend` Just "bill"

In [ ]:
-- Foldable are for things that can be folded up
import qualified Data.Foldable as F

-- whereas foldr takes a list, F.foldr takes any instance of foldable
:t foldr
:t F.foldr

-- so now we can fold over things like maybe
F.foldr (+) 1 $ Just 9

-- a more usefull example would be folding a tree
data Tree a = Empty | Node a (Tree a) (Tree a) deriving (Show, Read, Eq)

-- we could implement foldl and foldr
-- but its easier to implemtn foldmap
-- foldmap takes a value of the inner type we are folding and returns a monoid value
-- the second parameter is a foldable datastructure that contains the monoid values
-- then by mappending the values we can fold any structure

-- heres how it would look for a tree
instance F.Foldable Tree where
    foldMap f Empty = mempty
    foldMap f (Node x l r) = F.foldMap f l `mappend`
                             f x           `mappend`
                             F.foldMap f r
                             
-- foldMap receives a function that turns a Node into a monoid value
-- all that is needed is to fold the left and right trees
-- and combine them into a single monoid value for that node

let testTree = Node 5  
                (Node 3  
                    (Node 1 Empty Empty)  
                    (Node 6 Empty Empty)  
                )  
                (Node 9  
                    (Node 8 Empty Empty)  
                    (Node 10 Empty Empty)  
                )

F.foldl (+) 0 testTree
F.foldr (*) 1 testTree

-- one intresting use of foldMap is to turn trees to lists
F.foldMap (\x -> [x]) testTree